# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
url = 'https://en.wikipedia.org/wiki/Data_science'

In [3]:
res = requests.get("https://en.wikipedia.org/wiki/Data_science")


## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [4]:
soup = BeautifulSoup(res.text, "html.parser")
#soup

In [5]:
html = urlopen("https://en.wikipedia.org/wiki/Data_science")
links_html = BeautifulSoup(html)
links = []
for link in links_html.findAll("a"):
    if 'href' in link.attrs:
           links.append( link.attrs['href'])
        


In [6]:
len(links)
#links

457

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [10]:
domain = 'http://wikipedia.org'

In [8]:
#vodka = []
#for l in links:
#    if 'http' in l:
#        vodka.append(l)
        
vodka =[l for l in links if 'http' in l]
#vodka

In [11]:
#relative = []
#for r in links:
#    if '/wiki/' in r:
#        relative.append(r)
relative =[r for r in links if '/wiki/'in r]
        
relative_updated = [domain + x for x in relative]
#relative_updated
        


In [9]:
vodka_s = set(vodka)
relative_u_s = set(relative_updated)
combined = list(vodka_s) + list(relative_u_s)
#combined

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [10]:
import os

In [11]:
#os.mkdir('wikipedia')
os.chdir("wikipedia")

In [12]:
os.getcwd()

'/Users/esdras/Ironhack/datamex0320/module-1/lab-parallelization/your-code/wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [13]:
from slugify import slugify

In [14]:
def index_page(link):
    try:
        resp = requests.get(link).content
        soup = BeautifulSoup(resp, 'lxml')
        nombre = str(link)+".html"
        nombre = nombre.replace("/","-")
        with open (nombre, "w+") as f:
            s = f.write(str(soup))
        return s
    except: pass
        
        
    

In [15]:
index_page('https://en.wikipedia.org')

78500

In [16]:
nombre = str('https://en.wikipedia.org')+".html"
nombre

'https://en.wikipedia.org.html'

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [17]:
import time
from tqdm import tqdm
from tqdm import trange

In [18]:
%%time
for i in trange(len(combined)):
    index_page(combined[i])

100%|██████████| 296/296 [10:33<00:00,  2.14s/it] 

CPU times: user 1min 6s, sys: 2.88 s, total: 1min 9s
Wall time: 10min 33s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [27]:
import multiprocessing as mp

In [28]:
mp.cpu_count()

4

In [30]:
%%time
pool = mp.Pool()
pool.map(index_page, combined)
pool.close()

CPU times: user 120 ms, sys: 50.6 ms, total: 170 ms
Wall time: 2min 20s
